In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit

#  1. Data Cleaning and Preperation

In [ ]:
value_added = pd.read_csv('value_added_data.csv',thousands=",")

In [ ]:
value_added

,2012,2012.1,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Space economy1,101966,111559,113173,121868,123054,122183,119387,122860,118004,111560,115265,115962
1,Private industries,79861,89170,90495,98536,99506,98699,95196,96575,92167,85568,87779,86351
2,Oil and gas ext,39,30,38,32,18,6,3,3,1,1,1,1
3,"Mining, except oil and gas",0,0,0,1,1,1,0,0,0,0,0,0
4,Support activities for mining,14,10,13,12,7,3,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,"Performing arts, spectator sports, museums, an...",79,85,93,82,93,99,102,103,106,84,75,89
63,"Amusements, gambling, and recreation industries",0,0,0,0,0,0,0,0,0,0,0,0
64,Accommodation,3,4,4,4,3,3,3,3,2,2,3,5
65,Food services and drinking places,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
value_added = value_added.dropna(how="all")

In [ ]:
value_added = value_added.T

In [ ]:
value_added

,0,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,62,63,64,65,66
2012,Space economy1,Private industries,Oil and gas ext,"Mining, except oil and gas",Support activities for mining,Utilities,Construction,Wood products,Nonmetallic mineral products,Primary metals,...,Educational services,Ambulatory health care services,Hospitals,Nursing and residential care facilities,Social assistance,"Performing arts, spectator sports, museums, an...","Amusements, gambling, and recreation industries",Accommodation,Food services and drinking places,"Other services, except government"
2012.1,101966,79861,39,0,14,1,265,21,53,50,...,1641,2,76,0,0,79,0,3,0,3
2013,111559,89170,30,0,10,1,239,12,63,53,...,1558,2,76,0,0,85,0,4,0,4
2014,113173,90495,38,0,13,1,346,19,66,82,...,1473,2,62,0,0,93,0,4,0,4
2015,121868,98536,32,1,12,2,406,5,27,68,...,1599,2,71,0,0,82,0,4,0,4
2016,123054,99506,18,1,7,2,403,5,29,49,...,1515,4,75,0,0,93,0,3,0,5
2017,122183,98699,6,1,3,2,585,4,27,45,...,1424,4,78,0,0,99,0,3,0,5
2018,119387,95196,3,0,1,2,360,5,27,32,...,1609,3,83,0,0,102,0,3,0,4
2019,122860,96575,3,0,1,1,1049,3,25,28,...,1748,4,68,0,0,103,0,3,0,5
2020,118004,92167,1,0,1,1,850,3,23,30,...,1990,3,35,0,0,106,0,2,0,5


In [ ]:
value_added.columns = value_added.iloc[0]
value_added = value_added.drop(columns=["Private industries"])
value_added = value_added.drop(value_added.index[0])
value_added

2012,Space economy1,Oil and gas ext,"Mining, except oil and gas",Support activities for mining,Utilities,Construction,Wood products,Nonmetallic mineral products,Primary metals,Fabricated metal products,...,Educational services,Ambulatory health care services,Hospitals,Nursing and residential care facilities,Social assistance,"Performing arts, spectator sports, museums, and related activities","Amusements, gambling, and recreation industries",Accommodation,Food services and drinking places,"Other services, except government"
2012.1,101966,39,0,14,1,265,21,53,50,260,...,1641,2,76,0,0,79,0,3,0,3
2013,111559,30,0,10,1,239,12,63,53,443,...,1558,2,76,0,0,85,0,4,0,4
2014,113173,38,0,13,1,346,19,66,82,324,...,1473,2,62,0,0,93,0,4,0,4
2015,121868,32,1,12,2,406,5,27,68,363,...,1599,2,71,0,0,82,0,4,0,4
2016,123054,18,1,7,2,403,5,29,49,323,...,1515,4,75,0,0,93,0,3,0,5
2017,122183,6,1,3,2,585,4,27,45,266,...,1424,4,78,0,0,99,0,3,0,5
2018,119387,3,0,1,2,360,5,27,32,180,...,1609,3,83,0,0,102,0,3,0,4
2019,122860,3,0,1,1,1049,3,25,28,168,...,1748,4,68,0,0,103,0,3,0,5
2020,118004,1,0,1,1,850,3,23,30,163,...,1990,3,35,0,0,106,0,2,0,5
2021,111560,1,0,1,2,513,7,23,24,204,...,2030,4,76,0,0,84,0,2,0,6


In [ ]:
X = value_added.copy().drop(columns=["Space economy1"])
features = X.columns
X = X.values
y = value_added["Space economy1"].values

# 2. Dataset Split and Time Series Split Cross Validation

In [ ]:
value_added = value_added.sort_index()

In [ ]:
tss = TimeSeriesSplit(n_splits=4, test_size=2, gap=0)

In [ ]:
for fold, (train_idx, test_idx) in enumerate(tss.split(X)):
    print(f"Fold {fold+1}")
    print(f"  Train years: {value_added.index[train_idx][0]} → {value_added.index[train_idx][-1]}")
    print(f"  Test years : {value_added.index[test_idx][0]} → {value_added.index[test_idx][-1]}")
    print()

Fold 1
  Train years: 2012.1 → 2015
  Test years : 2016 → 2017

Fold 2
  Train years: 2012.1 → 2017
  Test years : 2018 → 2019

Fold 3
  Train years: 2012.1 → 2019
  Test years : 2020 → 2021

Fold 4
  Train years: 2012.1 → 2021
  Test years : 2022 → 2023



In [ ]:
years = value_added.index

# 3. Train and Evaluate different models

In [ ]:
import sklearn.metrics as metrics

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge, ElasticNet
from xgboost import XGBRegressor

In [ ]:
def evaluate(y_true, y_pred):
    rmse = metrics.root_mean_squared_error(y_true, y_pred)
    mae = metrics.mean_absolute_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100  # 小心 0

    return {"RMSE": rmse, "MAE": mae, "R2": r2, "MAPE": mape}


### Ridge

In [ ]:
ridge_fold_metrics = []
ridge_coefs = []

In [ ]:
for fold, (train_idx, test_idx) in enumerate(tss.split(X), 1):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]


    scaler = StandardScaler().fit(X_train)
    X_train_s = scaler.transform(X_train)
    X_test_s  = scaler.transform(X_test)

    model = Ridge(alpha=150, random_state=42)
    model.fit(X_train_s, y_train)
    y_pred = model.predict(X_test_s)

    m = evaluate(y_test, y_pred)
    ridge_fold_metrics.append(m)
    ridge_coefs.append(np.abs(model.coef_))

    print(f"Ridge | Fold {fold}: {m}")


Ridge | Fold 1: {'RMSE': 4553.42618431845, 'MAE': 4136.686152345363, 'R2': -108.32016601284077, 'MAPE': np.float64(3.3681535223441945)}
Ridge | Fold 2: {'RMSE': 3209.181996382998, 'MAE': 3019.744198990491, 'R2': -2.415380692406127, 'MAPE': np.float64(2.4807640927996424)}
Ridge | Fold 3: {'RMSE': 6611.154884619249, 'MAE': 5782.9353559348965, 'R2': -3.21020838158603, 'MAPE': np.float64(5.1205783320050475)}
Ridge | Fold 4: {'RMSE': 1640.5575566639122, 'MAE': 1632.746286135858, 'R2': -21.160388932498705, 'MAPE': np.float64(1.4126751007856355)}


In [ ]:
ridge_mean = {k: np.mean([fm[k] for fm in ridge_fold_metrics]) for k in ridge_fold_metrics[0]}
print("\nRidge | Mean over folds:", ridge_mean)

ridge_importance = pd.Series(np.mean(np.vstack(ridge_coefs), axis=0), index=features)\
                    .sort_values(ascending=False)
print("\nRidge | Top feature importance:")
print(ridge_importance.head(15))


Ridge | Mean over folds: {'RMSE': np.float64(4003.5801554961527), 'MAE': np.float64(3643.027998351652), 'R2': np.float64(-33.776536004832906), 'MAPE': np.float64(3.09554276198363)}

Ridge | Top feature importance:
2012
Wholesale trade                                                         171.031361
Data processing, internet publishing, and other information services    164.128022
Wood products                                                           148.628444
Petroleum and coal products                                             147.124299
Machinery                                                               142.291727
Administrative and support services                                     142.111550
Printing and related support activities                                 136.238315
Mining, except oil and gas                                              121.771440
Real estate                                                             120.969135
Other retail                     

### Plastic Net

In [ ]:
enet_fold_metrics = []
enet_coefs = []

In [ ]:
for fold, (train_idx, test_idx) in enumerate(tss.split(X), 1):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    scaler = StandardScaler().fit(X_train)
    X_train_s = scaler.transform(X_train)
    X_test_s  = scaler.transform(X_test)


    model = ElasticNet(alpha=100, l1_ratio=0.5, random_state=42, max_iter=10000)
    model.fit(X_train_s, y_train)
    y_pred = model.predict(X_test_s)

    m = evaluate(y_test, y_pred)
    enet_fold_metrics.append(m)
    enet_coefs.append(np.abs(model.coef_))

    print(f"ElasticNet | Fold {fold}: {m}")


ElasticNet | Fold 1: {'RMSE': 5466.858599347909, 'MAE': 5209.121629791749, 'R2': -156.57937124549915, 'MAPE': np.float64(4.243483475994031)}
ElasticNet | Fold 2: {'RMSE': 3876.240449024323, 'MAE': 3653.855076447595, 'R2': -3.9827814963019925, 'MAPE': np.float64(3.00193607338993)}
ElasticNet | Fold 3: {'RMSE': 5477.277968158856, 'MAE': 4394.988344545309, 'R2': -1.889871227921172, 'MAPE': np.float64(3.912008819967446)}
ElasticNet | Fold 4: {'RMSE': 1607.1409733894325, 'MAE': 1574.7424967995103, 'R2': -20.26681151108467, 'MAPE': np.float64(1.3629244365274753)}


In [ ]:
enet_mean = {k: np.mean([fm[k] for fm in enet_fold_metrics]) for k in enet_fold_metrics[0]}
print("\nElasticNet | Mean over folds:", enet_mean)


enet_importance = pd.Series(np.mean(np.vstack(enet_coefs), axis=0), index=features)\
                   .sort_values(ascending=False)
print("\nElasticNet | Top feature importance:")
print(enet_importance.head(15))


ElasticNet | Mean over folds: {'RMSE': np.float64(4106.87949748013), 'MAE': np.float64(3708.1768868960407), 'R2': np.float64(-45.679708870201736), 'MAPE': np.float64(3.1300882014697207)}

ElasticNet | Top feature importance:
2012
Wholesale trade                                                         94.946622
Data processing, internet publishing, and other information services    94.088061
Administrative and support services                                     86.359026
Wood products                                                           86.175109
Machinery                                                               84.823945
Petroleum and coal products                                             82.932887
Printing and related support activities                                 78.090412
Real estate                                                             77.937309
Computer and electronic products2                                       72.598685
Other retail                   

XGboost

In [ ]:
xgb_fold_metrics = []
xgb_importances = []
feature_names = features
feature_names

Index(['Oil and gas ext', 'Mining, except oil and gas',
       'Support activities for mining', 'Utilities', 'Construction',
       'Wood products', 'Nonmetallic mineral products', 'Primary metals',
       'Fabricated metal products', 'Machinery',
       'Computer and electronic products2',
       'Electrical equipment, appliances, and components',
       'Motor vehicles, bodies and trailers, and parts',
       'Other transportation equipment3', 'Furniture and related products',
       'Miscellaneous manufacturing', 'Food and beverage and tobacco products',
       'Textile mills and textile product mills',
       'Apparel and leather and allied products', 'Paper products',
       'Printing and related support activities',
       'Petroleum and coal products', 'Chemical products',
       'Plastics and rubber products', 'Wholesale trade',
       'Motor vehicle and parts dealers', 'Food and beverage stores',
       'General merchandise stores', 'Other retail', 'Air transportation',
      

In [ ]:
for fold, (train_idx, test_idx) in enumerate(tss.split(X)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    model = xgb.XGBRegressor(
        n_estimators=400,
        learning_rate=0.15,
        max_depth=6,
        random_state=42
    )
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    m = evaluate(y_test, y_pred)
    xgb_fold_metrics.append(m)
    print(f"[XGB] Fold {fold}: {m}")

    gain_dict = model.get_booster().get_score(importance_type='gain')
    imp_vec = np.array([gain_dict.get(col, 0.0) for col in feature_names], dtype=float)
    xgb_importances.append(imp_vec)


[XGB] Fold 0: {'RMSE': 867.75146484375, 'MAE': 750.5546875, 'R2': -2.9702184200286865, 'MAPE': np.float64(0.6108518577222227)}
[XGB] Fold 1: {'RMSE': 6022.66845703125, 'MAE': 5740.0234375, 'R2': -11.028966903686523, 'MAPE': np.float64(4.718371911400783)}
[XGB] Fold 2: {'RMSE': 8539.27734375, 'MAE': 7780.0859375, 'R2': -6.024107456207275, 'MAPE': np.float64(6.869634689779197)}
[XGB] Fold 3: {'RMSE': 3842.000732421875, 'MAE': 3827.34765625, 'R2': -120.53722381591797, 'MAPE': np.float64(3.311371894758027)}


In [ ]:
xgb_mean = np.mean(np.vstack(xgb_importances), axis=0)
xgb_importance = pd.Series(xgb_mean, index=feature_names).sort_values(ascending=False)

print("\n[XGB] Mean over folds:", {k: np.mean([fm[k] for fm in xgb_fold_metrics]) for k in xgb_fold_metrics[0]})
print("\n[XGB] Top feature importance (gain):")
print(xgb_importance.head(15))


[XGB] Mean over folds: {'RMSE': np.float64(4817.924499511719), 'MAE': np.float64(4524.5029296875), 'R2': np.float64(-35.14012914896011), 'MAPE': np.float64(3.877557588415057)}

[XGB] Top feature importance (gain):
2012
Oil and gas ext                                     0.0
Mining, except oil and gas                          0.0
Support activities for mining                       0.0
Utilities                                           0.0
Construction                                        0.0
Wood products                                       0.0
Nonmetallic mineral products                        0.0
Primary metals                                      0.0
Fabricated metal products                           0.0
Machinery                                           0.0
Computer and electronic products2                   0.0
Electrical equipment, appliances, and components    0.0
Motor vehicles, bodies and trailers, and parts      0.0
Other transportation equipment3                     

In [ ]:
from sklearn.cross_decomposition import PLSRegression

In [ ]:
pls_fold_metrics = []
pls_coefs = []

In [ ]:
for fold, (train_idx, test_idx) in enumerate(tss.split(X), 1):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    pls = PLSRegression(n_components=2, scale=True)  # n_components 可改 1/2/3/4 试
    pls.fit(X_train, y_train)

    y_pred = pls.predict(X_test).ravel()
    m = evaluate(y_test, y_pred)
    pls_fold_metrics.append(m)
    pls_coefs.append(np.abs(pls.coef_.ravel()))

    print(f"[PLS] Fold {fold}: {m}")


[PLS] Fold 1: {'RMSE': 6136.490851076193, 'MAE': 4767.798236604889, 'R2': -197.54724416603807, 'MAPE': np.float64(3.899557971269416)}
[PLS] Fold 2: {'RMSE': 3092.8393549083667, 'MAE': 3051.1551898765756, 'R2': -2.1722335248188718, 'MAPE': np.float64(2.5135713743879036)}
[PLS] Fold 3: {'RMSE': 7603.798472897293, 'MAE': 7079.036866366303, 'R2': -4.569421972892294, 'MAPE': np.float64(6.2401758886727725)}
[PLS] Fold 4: {'RMSE': 446.1464970114318, 'MAE': 445.6318492686987, 'R2': -0.6388885079985882, 'MAPE': np.float64(0.38539731164300534)}


In [ ]:
pls_mean = {k: float(np.mean([fm[k] for fm in pls_fold_metrics])) for k in pls_fold_metrics[0]}
print("\n[PLS] Mean over folds:", pls_mean)
pls_importance = pd.Series(np.mean(np.vstack(pls_coefs), axis=0),
                           index=features).sort_values(ascending=False)
print("\n[PLS] Top feature importance:")
print(pls_importance.head(15))


[PLS] Mean over folds: {'RMSE': 4319.818793973322, 'MAE': 3835.9055355291166, 'R2': -51.23194704293695, 'MAPE': 3.259675636493274}

[PLS] Top feature importance:
2012
Printing and related support activities                         729.425343
Transit and ground passenger transportation                     671.789290
Mining, except oil and gas                                      653.742225
Utilities                                                       513.490506
Other services, except government                               394.867507
Accommodation                                                   374.537137
Warehousing and storage                                         355.135184
Rental and leasing services and lessors of intangible assets    318.632747
Other real estate                                               245.135230
Real estate                                                     239.973074
Housing                                                         225.988656
Motor v

In [ ]:
from sklearn.linear_model import BayesianRidge

In [ ]:
br_fold_metrics = []
br_coefs = []

In [ ]:
for fold, (train_idx, test_idx) in enumerate(tss.split(X), 1):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    br = BayesianRidge()
    br.fit(X_train, y_train)

    y_pred = br.predict(X_test)
    m = evaluate(y_test, y_pred)
    br_fold_metrics.append(m)
    br_coefs.append(np.abs(br.coef_.ravel()))

    print(f"[BayesianRidge] Fold {fold}: {m}")


[BayesianRidge] Fold 1: {'RMSE': 777.5997006824676, 'MAE': 706.9854961501987, 'R2': -2.1881287433790853, 'MAPE': np.float64(0.5756427008472838)}
[BayesianRidge] Fold 2: {'RMSE': 1183.0026306333777, 'MAE': 1064.4975094223628, 'R2': 0.535889017541186, 'MAPE': np.float64(0.8729238905146511)}
[BayesianRidge] Fold 3: {'RMSE': 2460.346884841261, 'MAE': 1928.3090871339082, 'R2': 0.4169019175520007, 'MAPE': np.float64(1.718698723733494)}
[BayesianRidge] Fold 4: {'RMSE': 6851.842472876484, 'MAE': 6745.989599099004, 'R2': -385.55311262750695, 'MAPE': np.float64(5.831874677524876)}


In [ ]:
br_mean = {k: float(np.mean([fm[k] for fm in br_fold_metrics])) for k in br_fold_metrics[0]}
print("\n[BayesianRidge] Mean over folds:", br_mean)

br_importance = pd.Series(np.mean(np.vstack(br_coefs), axis=0),
                          index=features).sort_values(ascending=False)
print("\n[BayesianRidge] Top feature importance:")
print(br_importance.head(15))


[BayesianRidge] Mean over folds: {'RMSE': 2818.1979222583977, 'MAE': 2611.4454229513685, 'R2': -96.69711260894822, 'MAPE': 2.2497849981550764}

[BayesianRidge] Top feature importance:
2012
Computer and electronic products2                                 1.232687
Broadcasting and telecommunications4                              1.090246
Wholesale trade                                                   1.047689
Other transportation equipment3                                   1.041774
Construction                                                      1.032695
Computer systems design and related services                      0.987014
Miscellaneous professional, scientific, and technical services    0.823492
Other retail                                                      0.797417
Management of companies and enterprises                           0.600456
Insurance carriers and related activities                         0.457193
Electrical equipment, appliances, and components            

### 4. Analyzing Feature Importance

In [ ]:
model_importances = {
    "Ridge": ridge_importance,
    "ElasticNet": enet_importance,
    "PLS": pls_importance,
    "BayesianRidge": br_importance,
    "XGB": xgb_importance,
}

In [ ]:
def mean_mae(fold_metrics):
    return float(np.mean([m['MAE'] for m in fold_metrics]))
model_mae = {
    "Ridge":        mean_mae(ridge_fold_metrics),
    "ElasticNet":   mean_mae(enet_fold_metrics),
    "PLS":          mean_mae(pls_fold_metrics),
    "BayesianRidge":mean_mae(br_fold_metrics),
    "XGB":          mean_mae(xgb_fold_metrics),
}
model_mae

{'Ridge': 3643.027998351652,
 'ElasticNet': 3708.1768868960407,
 'PLS': 3835.9055355291166,
 'BayesianRidge': 2611.4454229513685,
 'XGB': 4524.5029296875}

In [ ]:
scaled_importances = {}
for name, s in model_importances.items():
    if s is None or len(s) == 0:
        continue
    s = s.abs()
    maxv = s.max()
    if maxv is None or maxv == 0:
        continue
    scaled_importances[name] = s / maxv
weights = {}

In [ ]:

for name, s in scaled_importances.items():
    mae = model_mae.get(name, None)
    if mae is None or mae <= 0:
        continue
    weights[name] = 1.0 / mae

In [ ]:
if len(weights) == 0:
    raise ValueError("No model")
w_sum = sum(weights.values())
weights = {k: v / w_sum for k, v in weights.items()}

print("model weight")
for k, v in weights.items():
    print(f"  {k:14s}: {v:.3f}")

model weight
  Ridge         : 0.231
  ElasticNet    : 0.227
  PLS           : 0.219
  BayesianRidge : 0.322


In [ ]:
all_features = sorted(set().union(*[s.index for s in scaled_importances.values()]))
stack = pd.DataFrame({name: scaled_importances[name].reindex(all_features).fillna(0.0)
                      for name in scaled_importances.keys()})

weighted_importance = stack.mul(pd.Series(weights), axis=1).sum(axis=1).sort_values(ascending=False)

In [ ]:
print("Overall Importance Top15")
print(weighted_importance.head(15))


Overall Importance Top15
2012
Wholesale trade                                                         0.732188
Computer and electronic products2                                       0.645242
Printing and related support activities                                 0.591082
Construction                                                            0.544529
Other retail                                                            0.536258
Mining, except oil and gas                                              0.530835
Broadcasting and telecommunications4                                    0.509836
Miscellaneous professional, scientific, and technical services          0.507505
Transit and ground passenger transportation                             0.480387
Data processing, internet publishing, and other information services    0.476279
Machinery                                                               0.470606
Utilities                                                               0.46088

In [ ]:
weighted_importance

,0
2012,
Wholesale trade,0.732188
Computer and electronic products2,0.645242
Printing and related support activities,0.591082
Construction,0.544529
Other retail,0.536258
...,...
"Amusements, gambling, and recreation industries",0.000000
Food and beverage stores,0.000000
Nursing and residential care facilities,0.000000


In [ ]:
import pandas as pd
import textwrap
import plotly.express as px

def wrap_label(txt, n=22, max_lines=3):
    lines = textwrap.wrap(str(txt), n)
    if len(lines) > max_lines:
        lines = lines[:max_lines-1] + [lines[max_lines-1] + "…"]
    return "<br>".join(lines)

# Convert Series to DataFrame
top15 = weighted_importance.reset_index()
top15.columns = ["industry", "importance"]

# Sort and keep top 15
top15 = top15.sort_values("importance", ascending=False).head(15).copy()

# Add rank
top15["rank"] = range(1, len(top15)+1)

# Add wrapped label
top15["label"] = top15.apply(
    lambda r: f"{r['rank']}. {wrap_label(r['industry'], n=22)}", axis=1
)

# Plot
fig = px.bar(
    top15.iloc[::-1],
    x="importance",
    y="label",
    orientation="h",
    color="importance",
    color_continuous_scale="Turbo",
    text=top15.iloc[::-1]["importance"].round(3),
    title="Top 15 Overall Importance (Weighted)"
)

fig.update_traces(
    textposition="outside",
    texttemplate="%{x:.3f}",
    cliponaxis=False
)

fig.update_layout(
    width=1200,
    height=max(650, 42*len(top15)+180),
    margin=dict(l=380, r=120, t=70, b=40),
    coloraxis_showscale=False,
    xaxis_title="Importance (unitless, weighted)",
    yaxis_title="",
    uniformtext_minsize=10,
    uniformtext_mode="hide",
    plot_bgcolor='white'
)

xmax = top15["importance"].max()
fig.update_xaxes(range=[0, xmax*1.10], showgrid=True, gridcolor="rgba(0,0,0,0.08)")
fig.update_yaxes(automargin=True)

fig.show()